<a href="https://colab.research.google.com/github/KRISHNAKANTH2003/ML/blob/main/yolo_rubbish_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
%pip install ultralytics

In [103]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
from PIL import Image

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 34.0/107.7 GB disk)


In [104]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vencerlanz09/taco-dataset-yolo-format")
train = path + '/train'
valid = path + '/valid'
test = path + '/test'

print("Path to dataset files:", path)

print("Path to dataset files:", train)

Path to dataset files: /root/.cache/kagglehub/datasets/vencerlanz09/taco-dataset-yolo-format/versions/1
Path to dataset files: /root/.cache/kagglehub/datasets/vencerlanz09/taco-dataset-yolo-format/versions/1/train


In [105]:
import os
print(os.listdir(train))


['labels', 'images', 'labels.cache']


In [106]:
import yaml

data_yaml = dict(
    train =train,
    val =valid,
    test =test,
    nc =18,
    names =['Aluminium foil', 'Bottle cap', 'Bottle', 'Broken glass', 'Can',
            'Carton', 'Cigarette', 'Cup', 'Lid', 'Other litter', 'Other plastic',
            'Paper', 'Plastic bag - wrapper', 'Plastic container', 'Pop tab',
            'Straw', 'Styrofoam piece', 'Unlabeled litter']
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

%cat data.yaml


{names: [Aluminium foil, Bottle cap, Bottle, Broken glass, Can, Carton, Cigarette,
    Cup, Lid, Other litter, Other plastic, Paper, Plastic bag - wrapper, Plastic container,
    Pop tab, Straw, Styrofoam piece, Unlabeled litter], nc: 18, test: /root/.cache/kagglehub/datasets/vencerlanz09/taco-dataset-yolo-format/versions/1/test,
  train: /root/.cache/kagglehub/datasets/vencerlanz09/taco-dataset-yolo-format/versions/1/train,
  val: /root/.cache/kagglehub/datasets/vencerlanz09/taco-dataset-yolo-format/versions/1/valid}


In [107]:
names =['Aluminium foil', 'Bottle cap', 'Bottle', 'Broken glass', 'Can',
        'Carton', 'Cigarette', 'Cup', 'Lid', 'Other litter', 'Other plastic',
        'Paper', 'Plastic bag - wrapper', 'Plastic container', 'Pop tab',
        'Straw', 'Styrofoam piece', 'Unlabeled litter']
N=list(range(len(names)))
class_map=dict(zip(N,names))

In [108]:
model = YOLO("yolov8x.pt")

In [ ]:
!yolo task=detect mode=train model=yolov8x.pt data=data.yaml epochs=1 imgsz=512

Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=N

In [101]:
paths2=[]
for dirname, _, filenames in os.walk('runs/detect/train'):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            paths2+=[(os.path.join(dirname, filename))]
paths2=sorted(paths2)

In [19]:
for path in paths2:
    image = Image.open(path)
    image=np.array(image)
    plt.figure(figsize=(20,10))
    plt.imshow(image)
    plt.show()

In [ ]:
best_path0='runs/detect/train/weights/best.pt'
source0='datasets/test'

In [ ]:
ppaths=[]
for dirname, _, filenames in os.walk(source0):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            ppaths+=[(os.path.join(dirname, filename))]
ppaths=sorted(ppaths)
print(ppaths[0])
print(len(ppaths))

In [ ]:
model2 = YOLO(best_path0)

In [ ]:
!yolo task=detect mode=predict model={best_path0} conf=0.1 source={source0}

In [ ]:
results = model2.predict(source0,conf=0.1)
print(len(results))

In [ ]:
print((results[0].boxes.data))

In [ ]:
PBOX=pd.DataFrame(columns=range(6))
for i in range(len(results)):
    arri=pd.DataFrame(results[i].boxes.data.cpu().numpy()).astype(float)
    path=ppaths[i]
    file=path.split('/')[-1]
    arri=arri.assign(file=file)
    arri=arri.assign(i=i)
    PBOX=pd.concat([PBOX,arri],axis=0)
PBOX.columns=['x','y','x2','y2','confidence','class','file','i']
display(PBOX)

In [ ]:
PBOX['class']=PBOX['class'].apply(lambda x: class_map[int(x)])
PBOX=PBOX.reset_index(drop=True)
display(PBOX)
display(PBOX['class'].value_counts())

In [ ]:
def draw_box2(n0):

    ipath=ppaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath.split('/')[-1]

    if PBOX[PBOX['file']==file] is not None:
        box=PBOX[PBOX['file']==file]
        box=box.reset_index(drop=True)
        #display(box)

        for i in range(len(box)):
            label=box.loc[i,'class']
            x=int(box.loc[i,'x'])
            y=int(box.loc[i,'y'])
            x2=int(box.loc[i,'x2'])
            y2=int(box.loc[i,'y2'])
            #print(label,x,y,x2,y2)
            cv2.putText(image, f'{label}', (x, int(y-4)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0),2)
            cv2.rectangle(image,(x,y),(x2,y2),(0,255,0),2) #green

    #plt.imshow(image)
    #plt.show()

    return image

In [ ]:
def create_animation(ims):
    fig=plt.figure(figsize=(12,8))
    im=plt.imshow(cv2.cvtColor(ims[0],cv2.COLOR_BGR2RGB))
    text = plt.text(0.05, 0.05, f'Slide {0}', transform=fig.transFigure, fontsize=14, color='blue')
    plt.axis('off')
    plt.close()

    def animate_func(i):
        im.set_array(cv2.cvtColor(ims[i],cv2.COLOR_BGR2RGB))
        text.set_text(f'Slide {20+i}')
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000)

In [ ]:
images2=[]
for i in tqdm(range(len(ppaths))):
    images2+=[draw_box2(i)]

In [ ]:
create_animation(images2)